# Idea: We want to dynamically assign class-attributes without the need for meta-classes.

We will accomplish this using descriptors

## 2nd Idea: We want to have lookup thingies that work like dictionaries, except that they return a default value

```
a.x ⟺ a.x[default_key]
```

## Design Goal


- Create a decorator `@attribute` that allows the setting of class attributes in a simple manner that does not require a metaclass.

```
class MyClass:

    @attribute
    def foo(cls):
        return f"{cls.__name__}'s foo attribute!"
```

- By default this should behave like follows:
    - appear when calling `dir(MyClass)`
    - listed in `help(MyClass)`
    - available without difference in behaviour in both instance and class calls.
    - immuable?
    
Augmentations:
    - implement `setter`, `getter` and `deleter` just as with `property`
    - implement distinction between calling on class and instance
    - implement lazy computation akin to `cached_property`
    
```
class MyClass:

    @attribute
    def foo(cls):
        return f"{cls.__name__}'s foo attribute!"

    @foo.setter
    def foo(cls):
        ...
        
    @foo.getter
    def foo(cls):
        ...
          
    @foo.deleter
    def foo(cls):
        ...
        
    @foo.instance
    def foo(self):
        ...
```

In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
%run class-properties/property_in_pure_python.ipynb

In [ ]:
from time import sleep

In [ ]:
from functools import cache, cached_property

In [ ]:
class Property:
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc
        self._name = ""

    def __set_name__(self, owner, name):
        self._name = name

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError(f"unreadable attribute {self._name}")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError(f"can't set attribute {self._name}")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError(f"can't delete attribute {self._name}")
        self.fdel(obj)

    def getter(self, fget):
        prop = type(self)(fget, self.fset, self.fdel, self.__doc__)
        prop._name = self._name
        return prop

    def setter(self, fset):
        prop = type(self)(self.fget, fset, self.fdel, self.__doc__)
        prop._name = self._name
        return prop

    def deleter(self, fdel):
        prop = type(self)(self.fget, self.fset, fdel, self.__doc__)
        prop._name = self._name
        return prop

In [ ]:
class Attribute(property):
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        super().__init__(cache(fget), fset, fdel, doc)

    # def __init__(self, fget=None, fset=None, fdel=None, doc=None):
    #     super().__init__()
    #     # self._fget = fget
    #     # self._fset = fset
    #     # self._fdel = fdel
    #     # if doc is None and fget is not None:
    #     #     doc = fget.__doc__
    #     # self.__doc__ = doc
    #     # self._name = ''

    def __set_name__(self, owner, name):
        self._name = name

    def __get__(self, obj, objtype=None):
        print(f"__get__, {obj=}, {objtype=}")
        # if obj is None:
        #     return self
        # if self.fget is None:
        #     raise AttributeError(f'unreadable attribute {self._name}')
        return self.fget(objtype)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError(f"can't set attribute {self._name}")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError(f"can't delete attribute {self._name}")
        self.fdel(obj)

    def getter(self, fget):
        prop = type(self)(fget, self.fset, self.fdel, self.__doc__)
        prop._name = self._name
        return prop

    def setter(self, fset):
        prop = type(self)(self.fget, fset, self.fdel, self.__doc__)
        prop._name = self._name
        return prop

    def deleter(self, fdel):
        prop = type(self)(self.fget, self.fset, fdel, self.__doc__)
        prop._name = self._name
        return prop

In [ ]:
class MyClass:
    data = 42

    @Property
    def python_property(self):
        print(f"computing python_property...", end="")
        sleep(1)
        print(f"done")
        return 43

    @property
    def instance_property(self):
        print(f"computing instance_property...", end="")
        sleep(1)
        print(f"done")
        return 43

    @Attribute
    def custom_property(cls):
        print(f"computing custom_property...", end="")
        sleep(1)
        print(f"done")
        return f"{cls.__name__}'s hidden data: {cls.data}"

In [ ]:
help(MyClass())

In [ ]:
help(MyClass)

In [ ]:
dir(MyClass)

In [ ]:
MyClass.custom_property

In [ ]:
MyClass().custom_property

In [ ]:
MyClass.__dict__["custom_property"].fget(1, 2, 3, 4, 5)

In [ ]:
MyClass.__dict__["custom_property"].fget

In [ ]:
help(MyClass())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=4, floatmode="fixed", suppress=True)
rng = np.random.default_rng()

In [ ]:
class Property:
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    def __get__(self, obj, objtype=None):
        print("__get__ called!")
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __getitem__(self, key):
        print("__getitem__ called!", key)
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)

In [ ]:
class A:
    @Property
    def my_dict(self, key=None):
        return {
            None: "default",
            42: "The secret meaning of life, the universe and everything",
        }[key]

In [ ]:
a = A()

In [ ]:
help(A)

In [ ]:
class Attribute(property):
    ...

In [ ]:
class A:
    @Attribute
    def b(cls):
        return 2

In [ ]:
A.b

In [ ]:
a.my_dict[0]